# 处理流程
- 根据pspnet图片选择原图
- 将原图放入maskrcnn处理
- 处理后结果图像融合
- 计算概率和中心点，更新概率标签
- 输出结果测试

## 根据pspnet图片选择原图

In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import colorsys
import random
import time

%matplotlib inline

In [5]:
main_root = '/home/flyingbird/Flyingbird/image_pre_process'

save_path = main_root + '/original_image_5000'

psp_image = os.listdir('pspnet')

for i in psp_image:
    image_id = i.strip().split('.')[0]
    image_path = main_root + '/original_image/5000/' + image_id + '.jpg'
    shutil.copy(image_path, save_path)

## 将原图放入maskrcnn处理，保存npy矩阵

## 图像融合

In [9]:
mask_read_path = 'maskrcnn'

PSPNet = 'pspnet'

mask_save_path = 'image_fusion/5000_fusion'

read_list = os.listdir(mask_read_path)
read_list.sort()
PSP_list  = os.listdir(PSPNet)
PSP_list.sort()

### **检查数据长度是否一直** 

In [50]:
count = 0

for i in range(len(image_list)):
    image, Mask, N = read_image(image_list[i], mask_list[i])
    #print(i)
    #print(N)
    #print('*' * 20)
    if N:
        label_list = find_mask_label(read_list[i])
        if N != len(label_list):
            #print(i)
            #print(N)
            #print(label_list)
            #print(len(label_list))
            count += 1
print(count)

6


In [11]:
def find_mask_label(string):
    mask_label = string.split('[')[1].split(']')[0].split(' ')
    return [int(s) for s in mask_label if s.isdigit()]

def read_image(image, mask):
    image = os.path.join(PSPNet, image)
    #image = os.path.join('Orinigal_image', image)

    mask = os.path.join(mask_read_path, mask)

    image = cv2.imread(image, 1)
    Mask = np.load(mask)

    Mask.dtype = 'uint8'
    Mask = Mask + 0

    N = Mask.shape[2]
    
    return image, Mask, N

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

def apply_mask(image, mask, color, alpha=0.5):
    """
    Apply the given mask to the image.
    """
    for c in range(3):
        #image[:, :, c] = np.where(mask == 1, image[:, :, c] * (1 - alpha) + alpha * color[c] * 255, image[:, :, c])
        #image[:, :, c] = np.where(mask == 1, 0.5 * 255, image[:, :, c])
        image[:, :, c] = np.where(mask == 1, color[c] * 210, image[:, :, c])
    return image

def add_mask(N, Mask, image, colors):
    #colors = random_colors(3)
    for i in range(N):
        color = colors[i]
        mask = Mask[:, :, i]

        image = apply_mask(image, mask, color)
        
    return image

In [49]:
# 文件夹名列表
image_list = PSP_list
mask_list = read_list

mask_info = []

image_nums = 2000

# 读取文件
for i in range(len(image_list)):
    # 将mask信息保存在array中
    # image_label = []
    rgb_list = []
    
    rgb_list.append(image_list[i].split('.')[0])
    
    image, Mask, N = read_image(image_list[i], mask_list[i])
    
    #"""
    if N:
        mask_label_list = find_mask_label(mask_list[i])
        if N != len(mask_label_list):
            N -= 1
        #print('N = ', N)
        for j in range(N):
            for c in range(3):
                rgb_list.append(int(random_colors(N)[j][c] * 210))
                
            #if j != 41: 
            #print(j)
            #print(mask_list[i])
            #print('changdu = ', len(find_mask_label(mask_list[i])))
                
            rgb_list.append(mask_label_list[j])
                
            #rgb_list.append(rgb_label)
    mask_info.append(rgb_list)
    #show_image(image)
    #"""
    # 加入掩码图像
    colors = random_colors(N)
    image = add_mask(N, Mask, image, colors)

    # 保存到对应路径
    save_path = os.path.join(mask_save_path, image_list[i])
    cv2.imwrite(save_path, image)
    if i % 100 == 0:
        #print("the %d image named %s has been saved" % (i, image_list[i]))
        print("%d images has been saved" % (i))
    if i == image_nums:
        print("process over")
        
    #mask_info.append(image_label)

mask_array = np.array([mask_info])
#mask_array.astype(np.int16)
np.save('mask_info.npy', mask_array)

0 images has been saved
100 images has been saved
200 images has been saved
300 images has been saved
400 images has been saved
500 images has been saved
600 images has been saved
700 images has been saved
800 images has been saved
900 images has been saved
1000 images has been saved
1100 images has been saved
1200 images has been saved
1300 images has been saved
1400 images has been saved
1500 images has been saved
1600 images has been saved
1700 images has been saved
1800 images has been saved
1900 images has been saved


## 计算概率和中心点，更新概率标签

In [29]:
# ava和psp的标签label查询表
ava_df = pd.read_csv('original_info/AVA.txt', sep=' ', header=None)
#psp_df = pd.read_csv('psp_label.txt', sep=' ', header=None)

# psp_list
psp_list = []
with open('original_info/psp_label.txt', 'r') as f:
    for i in f.readlines():
        psp_list.append((i.strip().split(' ')))
psp_list = np.array(psp_list).astype(np.int32)

In [30]:
# 输入图像路径
image_path = 'image_fusion/2000_fusion'

# 保存numpy.array路径
save_np = 'image_mask_psp_label_info/new_image_info.npy'

In [31]:
# 判断图像整体标签label
def choice_image_label(index):
    return ava_df[ava_df[1] == index].values[0][12:14]

# 判断部分图像标签part_label
def choice_part_label(array):
    for i in range(150):
        if (array[0] == psp_list[i][2]) and (array[1] == psp_list[i][1]) and (array[2] == psp_list[i][0]):
            return psp_list[i][3]
    return -1

In [32]:
def compute_mean(array, index_value):
    x_sum = 0
    y_sum = 0
    coor = np.array([])
    #for i in index:
    coordinate_array = np.argwhere(array == index_value)
    #print('对应坐标', coordinate_array)
    for i in coordinate_array:
        x_sum += i[0]
        y_sum += i[1]
            
    x_mean = x_sum // len(coordinate_array)
    y_mean = y_sum // len(coordinate_array)
    coor = np.append(coor, x_mean)
    coor = np.append(coor, y_mean)
    #print('平均坐标', coor)
    return coor

def compute_counts_and_coordinate(image):
    # 新建矩阵
    array = np.array([])
    
    # 将原图矩阵变形，转化为dataframe容易转换的shape
    shape = image.shape
    new_image = image.reshape([shape[0] * shape[1], 3])
    
    # pandas读取矩阵，并建立新的属性列
    df = pd.DataFrame(new_image)
    df[3] = df[0] + df[1] + df[2]
    #print(df.head())
    
    # 根据新的属性列分类，统计出现次数
    df2 = pd.DataFrame(df[3].value_counts())
    #print(df2.head())
          
    # 将变形后的图片还原为原来的尺寸
    np1 = np.array(df)
    np2 = np1.reshape([shape[0], shape[1], 4])
        # 根据统计次数取前5个，逐个计算索引的平均值，添加到image_info矩阵中
    for i in df2.index[:5]:
        # 添加rgb向量
        #print(df[df[3] == i].iloc[0][:3].values)
        rgb_array = df[df[3] == i].iloc[0][:3].values
        #print(array)
        array = np.append(array, rgb_array)
        
        # 添加平均值坐标向量
        coord = compute_mean(np2, i)
        array = np.append(array, coord)
        #print("添加坐标", array)
        
        # 添加出现次数
        array = np.append(array, df2.loc[i][3])
        #print(df2.loc[i][3])
        #print('添加次数', array)
        
        # 添加标签
        array = np.append(array, choice_part_label(rgb_array))
        
        #np.append(array)
        #part_image_info
        
        #array = array.astype(np.int32)
        
    return array

In [35]:
image_list = os.listdir(image_path)

image_info = []

#start_time = time.time()

for i in range(len(image_list)):
    start_time = time.time()
    
    image_array = np.array([])
    
    path = os.path.join(image_path, image_list[i])
    
    image_id = int(image_list[i].split('.')[0])
    #print(image_id)
    #filename = str(image_id) + '.png'
    #print(type(image_id))
    image = cv2.imread(path)
    
    whole_label = choice_image_label(image_id)
    
    part_image_info = compute_counts_and_coordinate(image)
    #shaped_image = image.reshape([480*640, 3])
    array = np.append(image_id, whole_label)
    array = np.append(array, part_image_info)
    array = array.astype(np.int32)
    #print(array.shape)

    #if not(i):
    #    image_info = np.append(image_info, array)
        #print(image_info.shape)
    #else:
        #image_info = np.vstack([image_info, array])
    image_info.append(array)
        
    if i % 20 == 0:
        print('%d images have processed'% i)
        #print('%f time have cost' % (time.time()-start_time))
    
#image_info = image_info.astype(np.int32)
#print(image_info.shape)
#print(time.time() - start_time)
#image_info
np.save(save_np, image_info)

0 images have processed
20 images have processed
40 images have processed
60 images have processed
80 images have processed
100 images have processed
120 images have processed
140 images have processed
160 images have processed
180 images have processed
200 images have processed
220 images have processed
240 images have processed
260 images have processed
280 images have processed
300 images have processed
320 images have processed
340 images have processed
360 images have processed
380 images have processed
400 images have processed
420 images have processed
440 images have processed
460 images have processed
480 images have processed
500 images have processed
520 images have processed
540 images have processed
560 images have processed
580 images have processed
600 images have processed
620 images have processed
640 images have processed
660 images have processed
680 images have processed
700 images have processed
720 images have processed
740 images have processed
760 images have pr

## 标签修改

In [55]:
# mask label信息
# image_info信息
# 将image_info信息中-1的值用对应的mask_label替换
image_info = np.load('image_mask_psp_label_info/new_image_info.npy')

mask_info = np.load('mask_info.npy')

In [68]:
for i in image_info:
    if -1 in i and i[0] != 103998 and i[0] != 252558 and i[0] != 347047:
        # 找到对应行
        for j in mask_info[0]:
            if int(j[0]) == i[0]:
                j = j[1:]
                list_mask = []

                for k in range(len(j)//4):
                    list_mask.append(j[(k+1)*4-1] + 150)
        
        # 提取-1位置和前面的rgb位置
        rgb_and_label_index = np.where(i == -1)
        for m in rgb_and_label_index:
            i[m] = list_mask[np.random.randint(len(list_mask))]

##  概率更新

In [69]:
# 读取数据集信息
image_array = np.load('image_mask_psp_label_info/new_image_info.npy')

In [70]:
pd_array = np.full([2000, 38], np.nan)

for i in range(2000):
    pd_array[i][:len(image_array[i])] = image_array[i]

columns = ['image_id', 'image_label_1', 'image_label_2', \
           'R_1', 'G_1', 'B_1', "X_1", "Y_1", 'Counts_1', 'part_label_1', \
           'R_2', 'G_2', 'B_2', "X_2", "Y_2", 'Counts_2', 'part_label_2', \
           'R_3', 'G_3', 'B_3', "X_3", "Y_3", 'Counts_3', 'part_label_3', \
           'R_4', 'G_4', 'B_4', "X_4", "Y_4", 'Counts_4', 'part_label_4', \
           'R_5', 'G_5', 'B_5', "X_5", "Y_5", 'Counts_5', 'part_label_5']
df = pd.DataFrame(pd_array, columns=columns)
df.head(5)

,image_id,image_label_1,image_label_2,R_1,G_1,B_1,X_1,Y_1,Counts_1,part_label_1,...,Y_4,Counts_4,part_label_4,R_5,G_5,B_5,X_5,Y_5,Counts_5,part_label_5
0,10334.0,15.0,19.0,230.0,230.0,6.0,223.0,281.0,257655.0,2.0,...,509.0,10911.0,3.0,3.0,200.0,4.0,138.0,528.0,1551.0,4.0
1,156455.0,15.0,0.0,140.0,140.0,140.0,186.0,306.0,201061.0,6.0,...,512.0,4022.0,34.0,210.0,0.0,0.0,189.0,401.0,3902.0,-1.0
2,321204.0,16.0,21.0,120.0,120.0,120.0,170.0,368.0,135109.0,0.0,...,510.0,25125.0,-1.0,210.0,0.0,0.0,241.0,230.0,1860.0,-1.0
3,145814.0,17.0,21.0,210.0,0.0,0.0,230.0,362.0,183512.0,-1.0,...,357.0,92.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38080.0,7.0,0.0,230.0,230.0,6.0,195.0,317.0,245718.0,2.0,...,309.0,689.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# 将对应label信息放入df中，计算value_counts
df_label = df[['image_label_1', 'image_label_2',\
              'part_label_1', 'part_label_2', 'part_label_3', 'part_label_4', 'part_label_5']]

df_label.head(5)

,image_label_1,image_label_2,part_label_1,part_label_2,part_label_3,part_label_4,part_label_5
0,15.0,19.0,2.0,12.0,0.0,3.0,4.0
1,15.0,0.0,6.0,-1.0,-1.0,34.0,-1.0
2,16.0,21.0,0.0,-1.0,12.0,-1.0,-1.0
3,17.0,21.0,-1.0,0.0,31.0,12.0,NaN
4,7.0,0.0,2.0,4.0,16.0,0.0,NaN


In [74]:
df_new_label1 = df_label[['image_label_1', 'image_label_2', 'part_label_1']] 
df_new_label2 = df_label[['image_label_1', 'image_label_2', 'part_label_2']] 
df_new_label3 = df_label[['image_label_1', 'image_label_2', 'part_label_3']] 
df_new_label4 = df_label[['image_label_1', 'image_label_2', 'part_label_4']] 
df_new_label5 = df_label[['image_label_1', 'image_label_2', 'part_label_5']] 

columns = ['image_label_1', 'image_label_2', 'part_label']
df_total_part_array = np.full([2000, 3], np.nan)
df_total_part_1 = pd.DataFrame(df_total_part_array, columns=columns)
df_total_part_2 = pd.DataFrame(df_total_part_array, columns=columns)
df_total_part_3 = pd.DataFrame(df_total_part_array, columns=columns)
df_total_part_4 = pd.DataFrame(df_total_part_array, columns=columns)
df_total_part_5 = pd.DataFrame(df_total_part_array, columns=columns)


df_total_part_1.iloc[:2000] = df_new_label1
df_total_part_2.iloc[:2000] = df_new_label2
df_total_part_3.iloc[:2000] = df_new_label3
df_total_part_4.iloc[:2000] = df_new_label4
df_total_part_5.iloc[:2000] = df_new_label5

df_total_part_1_prob = df_total_part_1['part_label'].value_counts()/2000
df_total_part_2_prob = df_total_part_2['part_label'].value_counts()/2000
df_total_part_3_prob = df_total_part_3['part_label'].value_counts()/2000
df_total_part_4_prob = df_total_part_4['part_label'].value_counts()/2000
df_total_part_5_prob = df_total_part_5['part_label'].value_counts()/2000

df_total_part_1['prob_v1'] = None
list_label = [df_total_part_1, df_total_part_2, df_total_part_3, \
              df_total_part_4, df_total_part_5]
list_prob = [df_total_part_1_prob, df_total_part_2_prob, df_total_part_3_prob, \
             df_total_part_4_prob, df_total_part_5_prob]

for m in range(5):
    for i in range(2000):
        for j in list_prob[m].index:
            if list_label[m].loc[i, 'part_label'] == j:
                list_label[m].loc[i, 'prob_v1'] = list_prob[m][j]
                if i*(m+1) % 100 == 0 and i != 0:
                    print('%d data has been modify'% i)

100 data has been modify
200 data has been modify
300 data has been modify
400 data has been modify
500 data has been modify
600 data has been modify
700 data has been modify
800 data has been modify
900 data has been modify
1000 data has been modify
1100 data has been modify
1200 data has been modify
1300 data has been modify
1400 data has been modify
1500 data has been modify
1600 data has been modify
1700 data has been modify
1800 data has been modify
1900 data has been modify
50 data has been modify
100 data has been modify
150 data has been modify
200 data has been modify
250 data has been modify
300 data has been modify
350 data has been modify
400 data has been modify
450 data has been modify
500 data has been modify
550 data has been modify
600 data has been modify
650 data has been modify
700 data has been modify
750 data has been modify
800 data has been modify
850 data has been modify
900 data has been modify
950 data has been modify
1000 data has been modify
1050 data has b

In [75]:
df_total_part_1.head(5)

,image_label_1,image_label_2,part_label,prob_v1
0,15.0,19.0,2.0,0.23
1,15.0,0.0,6.0,0.0035
2,16.0,21.0,0.0,0.2475
3,17.0,21.0,-1.0,0.23
4,7.0,0.0,2.0,0.23


In [80]:
def modify_label(df_label, df_prob):
    for i in range(2000): 
        for j in df_prob.index:
            if df_label.loc[i, 'part_label'] == j:
                df_cache = df_label.iloc[df_label[df_label['part_label'] == j]['prob_v1'].idxmax()]
                if df_label.loc[i, 'image_label_1'] == 0 and df_label.loc[i, 'image_label_2'] != 0:
                    df_label.loc[i, 'part_label'] = df_cache.loc['image_label_2']
                elif df_label.loc[i, 'image_label_1'] != 0 and df_label.loc[i, 'image_label_2'] == 0:
                    df_label.loc[i, 'part_label'] = df_cache.loc['image_label_1']
                elif df_label.loc[i, 'image_label_1'] != 0 and df_label.loc[i, 'image_label_2'] != 0:
                    df_label.loc[i, 'part_label'] = df_cache.loc[df_label.columns[random.randint(0,1)]]
                else:
                    df_label.loc[i, 'part_label'] = df_cache.loc['part_label']

In [81]:
modify_label(df_total_part_1, df_total_part_1_prob)
modify_label(df_total_part_2, df_total_part_2_prob)
modify_label(df_total_part_3, df_total_part_3_prob)
modify_label(df_total_part_4, df_total_part_4_prob)
modify_label(df_total_part_5, df_total_part_5_prob)

In [ ]:
def prob_update(df_label, df_label, df_prob):
    df_new_label1 = df_label[['image_label_1', 'image_label_2', 'part_label_1']] 
    df_new_label2 = df_label[['image_label_1', 'image_label_2', 'part_label_2']] 
    df_new_label3 = df_label[['image_label_1', 'image_label_2', 'part_label_3']] 
    df_new_label4 = df_label[['image_label_1', 'image_label_2', 'part_label_4']] 
    df_new_label5 = df_label[['image_label_1', 'image_label_2', 'part_label_5']] 

    columns = ['image_label_1', 'image_label_2', 'part_label']
    df_total_part_array = np.full([2000, 3], np.nan)
    df_total_part_1 = pd.DataFrame(df_total_part_array, columns=columns)
    df_total_part_2 = pd.DataFrame(df_total_part_array, columns=columns)
    df_total_part_3 = pd.DataFrame(df_total_part_array, columns=columns)
    df_total_part_4 = pd.DataFrame(df_total_part_array, columns=columns)
    df_total_part_5 = pd.DataFrame(df_total_part_array, columns=columns)

    df_total_part_1.iloc[:2000] = df_new_label1
    df_total_part_2.iloc[:2000] = df_new_label2
    df_total_part_3.iloc[:2000] = df_new_label3
    df_total_part_4.iloc[:2000] = df_new_label4
    df_total_part_5.iloc[:2000] = df_new_label5

    df_total_part_1_prob = df_total_part_1['part_label'].value_counts()/2000
    df_total_part_2_prob = df_total_part_2['part_label'].value_counts()/2000
    df_total_part_3_prob = df_total_part_3['part_label'].value_counts()/2000
    df_total_part_4_prob = df_total_part_4['part_label'].value_counts()/2000
    df_total_part_5_prob = df_total_part_5['part_label'].value_counts()/2000

    df_total_part_1['prob_v1'] = None
    list_label = [df_total_part_1, df_total_part_2, df_total_part_3, \
                  df_total_part_4, df_total_part_5]
    list_prob = [df_total_part_1_prob, df_total_part_2_prob, df_total_part_3_prob, \
                 df_total_part_4_prob, df_total_part_5_prob]

    for m in range(5):
        for i in range(2000):
            for j in list_prob[m].index:
                if list_label[m].loc[i, 'part_label'] == j:
                    list_label[m].loc[i, 'prob_v1'] = list_prob[m][j]
                    if i*(m+1) % 100 == 0 and i != 0:
                        print('%d data has been modify'% i)
                    
    modify_label(df_total_part_1, df_total_part_1_prob)
    modify_label(df_total_part_2, df_total_part_2_prob)
    modify_label(df_total_part_3, df_total_part_3_prob)
    modify_label(df_total_part_4, df_total_part_4_prob)
    modify_label(df_total_part_5, df_total_part_5_prob)

In [83]:
df_total_part_1_prob_v2 = df_total_part_1['part_label'].value_counts()/2000
df_total_part_2_prob_v2 = df_total_part_2['part_label'].value_counts()/2000
df_total_part_3_prob_v2 = df_total_part_3['part_label'].value_counts()/2000
df_total_part_4_prob_v2 = df_total_part_4['part_label'].value_counts()/2000
df_total_part_5_prob_v2 = df_total_part_5['part_label'].value_counts()/2000

df_total_part_1['prob_v2'] = None
df_total_part_2['prob_v2'] = None
df_total_part_3['prob_v2'] = None
df_total_part_4['prob_v2'] = None
df_total_part_5['prob_v2'] = None


list_label = [df_total_part_1, df_total_part_2, df_total_part_3, \
              df_total_part_4, df_total_part_5]
list_prob = [df_total_part_1_prob_v2, df_total_part_2_prob_v2, df_total_part_3_prob_v2, \
             df_total_part_4_prob_v2, df_total_part_5_prob_v2]

for m in range(5):
    for i in range(2000):
        for j in list_prob[m].index:
            if list_label[m].loc[i, 'part_label'] == j:
                list_label[m].loc[i, 'prob_v2'] = list_prob[m][j]
                if i*(m+1) % 100 == 0 and i != 0:
                    print('%d data has been modify'% i)
                    
modify_label(df_total_part_1, df_total_part_1_prob_v2)
modify_label(df_total_part_2, df_total_part_2_prob_v2)
modify_label(df_total_part_3, df_total_part_3_prob_v2)
modify_label(df_total_part_4, df_total_part_4_prob_v2)
modify_label(df_total_part_5, df_total_part_5_prob_v2)

100 data has been modify
200 data has been modify
300 data has been modify
400 data has been modify
500 data has been modify
600 data has been modify
700 data has been modify
800 data has been modify
900 data has been modify
1000 data has been modify
1100 data has been modify
1200 data has been modify
1300 data has been modify
1400 data has been modify
1500 data has been modify
1600 data has been modify
1700 data has been modify
1800 data has been modify
1900 data has been modify
50 data has been modify
100 data has been modify
150 data has been modify
200 data has been modify
250 data has been modify
300 data has been modify
350 data has been modify
400 data has been modify
450 data has been modify
500 data has been modify
550 data has been modify
600 data has been modify
650 data has been modify
700 data has been modify
750 data has been modify
800 data has been modify
850 data has been modify
900 data has been modify
950 data has been modify
1000 data has been modify
1050 data has b

In [84]:
df_total_part_1_prob_v2 = df_total_part_1['part_label'].value_counts()/2000
df_total_part_2_prob_v2 = df_total_part_2['part_label'].value_counts()/2000
df_total_part_3_prob_v2 = df_total_part_3['part_label'].value_counts()/2000
df_total_part_4_prob_v2 = df_total_part_4['part_label'].value_counts()/2000
df_total_part_5_prob_v2 = df_total_part_5['part_label'].value_counts()/2000

df_total_part_1['prob_v3'] = None
df_total_part_2['prob_v3'] = None
df_total_part_3['prob_v3'] = None
df_total_part_4['prob_v3'] = None
df_total_part_5['prob_v3'] = None

list_label = [df_total_part_1, df_total_part_2, df_total_part_3, \
              df_total_part_4, df_total_part_5]
list_prob = [df_total_part_1_prob_v2, df_total_part_2_prob_v2, df_total_part_3_prob_v2, \
             df_total_part_4_prob_v2, df_total_part_5_prob_v2]

for m in range(5):
    for i in range(2000):
        for j in list_prob[m].index:
            if list_label[m].loc[i, 'part_label'] == j:
                list_label[m].loc[i, 'prob_v3'] = list_prob[m][j]
                if i*(m+1) % 100 == 0 and i != 0:
                    print('%d data has been modify'% i)
                    
modify_label(df_total_part_1, df_total_part_1_prob_v2)
modify_label(df_total_part_2, df_total_part_2_prob_v2)
modify_label(df_total_part_3, df_total_part_3_prob_v2)
modify_label(df_total_part_4, df_total_part_4_prob_v2)
modify_label(df_total_part_5, df_total_part_5_prob_v2)

100 data has been modify
200 data has been modify
300 data has been modify
400 data has been modify
500 data has been modify
600 data has been modify
700 data has been modify
800 data has been modify
900 data has been modify
1000 data has been modify
1100 data has been modify
1200 data has been modify
1300 data has been modify
1400 data has been modify
1500 data has been modify
1600 data has been modify
1700 data has been modify
1800 data has been modify
1900 data has been modify
50 data has been modify
100 data has been modify
150 data has been modify
200 data has been modify
250 data has been modify
300 data has been modify
350 data has been modify
400 data has been modify
450 data has been modify
500 data has been modify
550 data has been modify
600 data has been modify
650 data has been modify
700 data has been modify
750 data has been modify
800 data has been modify
850 data has been modify
900 data has been modify
950 data has been modify
1000 data has been modify
1050 data has b

In [85]:
df_total_part_1.head(5)

,image_label_1,image_label_2,part_label,prob_v1,prob_v2,prob_v3
0,15.0,19.0,1.0,0.23,0.22,0.0775
1,15.0,0.0,15.0,0.0035,0.0105,0.0195
2,16.0,21.0,21.0,0.2475,0.0465,0.023
3,17.0,21.0,1.0,0.23,0.22,0.0775
4,7.0,0.0,7.0,0.23,0.0175,0.0065


In [86]:
df_total_part_1_prob_v2 = df_total_part_1['part_label'].value_counts()/2000
df_total_part_2_prob_v2 = df_total_part_2['part_label'].value_counts()/2000
df_total_part_3_prob_v2 = df_total_part_3['part_label'].value_counts()/2000
df_total_part_4_prob_v2 = df_total_part_4['part_label'].value_counts()/2000
df_total_part_5_prob_v2 = df_total_part_5['part_label'].value_counts()/2000

df_total_part_1['prob_v4'] = None
df_total_part_2['prob_v4'] = None
df_total_part_3['prob_v4'] = None
df_total_part_4['prob_v4'] = None
df_total_part_5['prob_v4'] = None

list_label = [df_total_part_1, df_total_part_2, df_total_part_3, \
              df_total_part_4, df_total_part_5]
list_prob = [df_total_part_1_prob_v2, df_total_part_2_prob_v2, df_total_part_3_prob_v2, \
             df_total_part_4_prob_v2, df_total_part_5_prob_v2]

for m in range(5):
    for i in range(2000):
        for j in list_prob[m].index:
            if list_label[m].loc[i, 'part_label'] == j:
                list_label[m].loc[i, 'prob_v4'] = list_prob[m][j]
                if i*(m+1) % 100 == 0 and i != 0:
                    print('%d data has been modify'% i)
                    
modify_label(df_total_part_1, df_total_part_1_prob_v2)
modify_label(df_total_part_2, df_total_part_2_prob_v2)
modify_label(df_total_part_3, df_total_part_3_prob_v2)
modify_label(df_total_part_4, df_total_part_4_prob_v2)
modify_label(df_total_part_5, df_total_part_5_prob_v2)

100 data has been modify
200 data has been modify
300 data has been modify
400 data has been modify
500 data has been modify
600 data has been modify
700 data has been modify
800 data has been modify
900 data has been modify
1000 data has been modify
1100 data has been modify
1200 data has been modify
1300 data has been modify
1400 data has been modify
1500 data has been modify
1600 data has been modify
1700 data has been modify
1800 data has been modify
1900 data has been modify
50 data has been modify
100 data has been modify
150 data has been modify
200 data has been modify
250 data has been modify
300 data has been modify
350 data has been modify
400 data has been modify
450 data has been modify
500 data has been modify
550 data has been modify
600 data has been modify
650 data has been modify
700 data has been modify
750 data has been modify
800 data has been modify
850 data has been modify
900 data has been modify
950 data has been modify
1000 data has been modify
1050 data has b

In [87]:
df_total_part_1.head(5)

,image_label_1,image_label_2,part_label,prob_v1,prob_v2,prob_v3,prob_v4
0,15.0,19.0,0.0,0.23,0.22,0.0775,0.108
1,15.0,0.0,15.0,0.0035,0.0105,0.0195,0.0175
2,16.0,21.0,16.0,0.2475,0.0465,0.023,0.0165
3,17.0,21.0,0.0,0.23,0.22,0.0775,0.108
4,7.0,0.0,7.0,0.23,0.0175,0.0065,0.004


In [88]:
df_total_part_1_prob_v2 = df_total_part_1['part_label'].value_counts()/2000
df_total_part_2_prob_v2 = df_total_part_2['part_label'].value_counts()/2000
df_total_part_3_prob_v2 = df_total_part_3['part_label'].value_counts()/2000
df_total_part_4_prob_v2 = df_total_part_4['part_label'].value_counts()/2000
df_total_part_5_prob_v2 = df_total_part_5['part_label'].value_counts()/2000

df_total_part_1['prob_v5'] = None
df_total_part_2['prob_v5'] = None
df_total_part_3['prob_v5'] = None
df_total_part_4['prob_v5'] = None
df_total_part_5['prob_v5'] = None

list_label = [df_total_part_1, df_total_part_2, df_total_part_3, \
              df_total_part_4, df_total_part_5]
list_prob = [df_total_part_1_prob_v2, df_total_part_2_prob_v2, df_total_part_3_prob_v2, \
             df_total_part_4_prob_v2, df_total_part_5_prob_v2]

for m in range(5):
    for i in range(2000):
        for j in list_prob[m].index:
            if list_label[m].loc[i, 'part_label'] == j:
                list_label[m].loc[i, 'prob_v5'] = list_prob[m][j]
                if i*(m+1) % 100 == 0 and i != 0:
                    print('%d data has been modify'% i)
                    
modify_label(df_total_part_1, df_total_part_1_prob_v2)
modify_label(df_total_part_2, df_total_part_2_prob_v2)
modify_label(df_total_part_3, df_total_part_3_prob_v2)
modify_label(df_total_part_4, df_total_part_4_prob_v2)
modify_label(df_total_part_5, df_total_part_5_prob_v2)

100 data has been modify
200 data has been modify
300 data has been modify
400 data has been modify
500 data has been modify
600 data has been modify
700 data has been modify
800 data has been modify
900 data has been modify
1000 data has been modify
1100 data has been modify
1200 data has been modify
1300 data has been modify
1400 data has been modify
1500 data has been modify
1600 data has been modify
1700 data has been modify
1800 data has been modify
1900 data has been modify
50 data has been modify
100 data has been modify
150 data has been modify
200 data has been modify
250 data has been modify
300 data has been modify
350 data has been modify
400 data has been modify
450 data has been modify
500 data has been modify
550 data has been modify
600 data has been modify
650 data has been modify
700 data has been modify
750 data has been modify
800 data has been modify
850 data has been modify
900 data has been modify
950 data has been modify
1000 data has been modify
1050 data has b

In [89]:
df_total_part_1.head(5)

,image_label_1,image_label_2,part_label,prob_v1,prob_v2,prob_v3,prob_v4,prob_v5
0,15.0,19.0,28.0,0.23,0.22,0.0775,0.108,0.139
1,15.0,0.0,15.0,0.0035,0.0105,0.0195,0.0175,0.0205
2,16.0,21.0,16.0,0.2475,0.0465,0.023,0.0165,0.013
3,17.0,21.0,17.0,0.23,0.22,0.0775,0.108,0.139
4,7.0,0.0,7.0,0.23,0.0175,0.0065,0.004,0.0035


In [90]:
df['part_label_1'] = df_total_part_1['part_label']
df['part_label_2'] = df_total_part_2['part_label']
df['part_label_3'] = df_total_part_3['part_label']
df['part_label_4'] = df_total_part_4['part_label']
df['part_label_5'] = df_total_part_5['part_label']

new_prob_update_array = df.values
new_prob_update_array.astype(np.int32)
np.save('prob_update_label.npy', new_prob_update_array)

## 输出结果测试

In [80]:
image_info_array = np.load('prob_update_label.npy')
old_image_array = np.load('image_mask_psp_label_info/new_image_info.npy')
#old_image_array.astype(image_info_array)

df = pd.read_csv('original_info/tags.txt', sep=' ', header=None)

In [81]:
def show_image(image):
    #for i in range(N):
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [89]:
input_image_id = input('please input image id :')
if input_image_id:
    print('input correct!')

image_id = int(input_image_id)
image_name = input_image_id + '.png'
image_file = os.path.join('image_fusion/2000_fusion', image_name)

image_array = cv2.imread(image_file)

please input image id :19939
input correct!


In [95]:
for i in range(len(image_info_array)):
    if image_info_array[i][0] == image_id:
        #print(i.astype(np.int32))
        if image_info_array[i][1] != 0:
        #print(i)
            print('image label 1 is: ', df[image_info_array[i][1] == df[0]].values[0][1])
        if image_info_array[i][2] != 0:
            print('image label 2 is: ', df[image_info_array[i][2] == df[0]].values[0][1])
        if len(image_info_array[i]) == 38: 
            for j in range(5):
                rgb = image_info_array[i][3+j*7 : 6+j*7]
                locate = image_info_array[i][6+j*7: 8+j*7]
                #if bayes_label_array[i[9+7*j]] != 0:
                #label = bayes_label_array[i[9+7*j]]
                #else:
                counts = image_info_array[i][8 + 7*j]
                label = image_info_array[i][9+7*j]
                print(label)
                print(rgb)
                old_label = old_image_array[i][9+7*j]
                #print(old_label)
                #print(label)
                #print(df[old_label == df[0]].values)
                if label and old_label != -1 and old_label != 0:
                    image_label = df[label == df[0]].values[0][1]
                    #print(df[old_label == df[0]].values[0][1])
                    #old_label = df[old_label == df[0]].values[0][1]
                    
                    print('{} rgb values x,y is {},aera is {}, new_label is: {} <== old_label is: {}'.format(rgb, locate, counts, image_label, old_label))
                

image label 1 is:  Candid
image label 2 is:  Travel
0.0
[210. 210.   0.]
17.0
[  3. 200.   4.]
[  3. 200.   4.] rgb values x,y is [159. 388.],aera is 53655.0, new_label is: Portraiture <== old_label is: 4
27.0
[230. 230.   6.]
[230. 230.   6.] rgb values x,y is [208. 321.],aera is 50574.0, new_label is: Rural <== old_label is: 2
40.0
[255.   0. 163.]
[255.   0. 163.] rgb values x,y is [329. 377.],aera is 11002.0, new_label is: Food_and_Drink <== old_label is: 116
23.0
[  7. 255. 235.]


In [96]:
show_image(image_array)